In [45]:
import pandas as pd

# 파일을 읽어옵니다.
file_name = 'data/elevator_lift.csv'
df = pd.read_csv(file_name)

# 숫자 열들을 숫자형으로 변환합니다.
numeric_columns = ['엘리베이터(E/V)', '에스컬레이터(E/S)', '휠체어리프트(W/L)', '수평자동보도(M/W)']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# 괄호로 시작하는 부분을 삭제합니다.
df['역명'] = df['역명'].str.replace(r'\(.*\)', '', regex=True).str.strip()

# '역명' 열을 인덱스로 설정합니다.
df.set_index('역명', inplace=True)

# '호선', '수평자동보도(M/W)', '에스컬레이터(E/S)' 열을 삭제합니다.
df.drop(columns=['호선', '수평자동보도(M/W)', '에스컬레이터(E/S)'], inplace=True)

# 역명으로 그룹화하고 수치를 평균 값으로 대체합니다.
df_grouped = df.groupby('역명').mean()

# 가중치를 설정합니다.
weight_elevator = 0.7
weight_wheelchair = 0.3

# 가중치를 적용하여 새로운 열을 생성합니다.
df_grouped['가중치_합'] = (df_grouped['엘리베이터(E/V)'] * weight_elevator +
                        df_grouped['휠체어리프트(W/L)'] * weight_wheelchair)

# 결과를 출력합니다.
df_grouped.drop(columns=['엘리베이터(E/V)', '휠체어리프트(W/L)'], inplace=True)

df_grouped

,가중치_합
역명,
가락시장,3.15
가산디지털단지,4.00
강남,2.80
강남구청,1.70
강동,2.00
...,...
홍제,2.10
화곡,1.40
화랑대,2.80


In [16]:
import pandas as pd

# 파일을 읽어옵니다.
file_name = 'data/elevator_lift.csv'
df = pd.read_csv(file_name)

# 숫자 열들을 숫자형으로 변환합니다.
numeric_columns = ['엘리베이터(E/V)', '에스컬레이터(E/S)', '휠체어리프트(W/L)', '수평자동보도(M/W)']
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

df

,호선,역명,엘리베이터(E/V),에스컬레이터(E/S),휠체어리프트(W/L),수평자동보도(M/W)
0,1호선,서울역(1),4,5,1,0
1,1호선,시청(1),3,3,0,0
2,1호선,종각,4,2,0,0
3,1호선,종로3가(1),3,4,0,0
4,1호선,종로5가,3,0,0,0
...,...,...,...,...,...,...
281,9호선(3단계),송파나루역,5,6,0,0
282,9호선(3단계),한성백제역,3,12,0,0
283,9호선(3단계),올림픽공원역,3,14,0,0
284,9호선(3단계),둔촌오륜역,3,8,0,0


In [18]:
import pandas as pd

# 파일을 읽어옵니다.
file_name = 'data/elevator_lift.csv'
df = pd.read_csv(file_name)

# 호선 열의 고유한 값들을 찾습니다.
unique_lines = df['호선'].unique()
unique_count = len(unique_lines)

# 고유한 값의 개수와 각각의 값을 출력합니다.
print(f"호선 열의 고유한 값 개수: {unique_count}")
print("고유한 값들:")
for line in unique_lines:
    print(line)


호선 열의 고유한 값 개수: 10
고유한 값들:
1호선
2호선
3호선
4호선
5호선
6호선
7호선
8호선
9호선(2단계)
9호선(3단계)


In [55]:
import pandas as pd

# CSV 파일을 'euc-kr' 인코딩으로 로드
file_path = 'data/subway1-8_name_lat_lon_gu.csv'
df = pd.read_csv(file_path, encoding='euc-kr')

# '역명' 열을 인덱스로 설정
df.set_index('역명', inplace=True)

# '위도'와 '경도' 열만 선택
df = df[['위도', '경도']]

# 결과 확인
print(df.head())


             위도          경도
역명                         
서울    37.553150  126.972533
시청    37.563590  126.975407
종각    37.570203  126.983116
종로3가  37.570429  126.992095
종로5가  37.570971  127.001900


In [56]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

# 서울시 행정구역 GeoJSON 데이터를 로드
geojson_path = 'data/HangJeongDong_ver20230701.geojson'
districts = gpd.read_file(geojson_path)

# 지하철 역 데이터를 GeoDataFrame으로 변환
df['geometry'] = df.apply(lambda row: Point(row['경도'], row['위도']), axis=1)
subway_gdf = gpd.GeoDataFrame(df, geometry='geometry')

# 좌표계 통일 (서울시 행정구역 GeoJSON 데이터와 동일한 좌표계를 사용해야 함)
subway_gdf.set_crs(epsg=4326, inplace=True)  # WGS84 좌표계

# 각 역의 좌표가 속한 구 찾기 (op 대신 predicate 사용)
subway_gdf = gpd.sjoin(subway_gdf, districts, how='left', predicate='within')

# 결과 데이터프레임에서 필요한 열만 선택
result_df = subway_gdf[['위도', '경도', 'sggnm']].copy()
result_df.rename(columns={'sggnm': '구'}, inplace=True)

# 인덱스로 그룹화하고 평균 계산
grouped_df = result_df.groupby(result_df.index).agg({
    '위도': 'mean',
    '경도': 'mean',
    '구': 'first'  # '구' 정보는 첫 번째 값을 사용
})

# 결과 출력
grouped_df

,위도,경도,구
역명,,,
가락시장,37.492686,127.118190,송파구
가산디지털단지,37.480376,126.882704,금천구
강남,37.497958,127.027539,서초구
강남구청,37.517185,127.041220,강남구
강동,37.535810,127.132490,강동구
...,...,...,...
홍제,37.588851,126.944092,서대문구
화곡,37.541585,126.840436,강서구
화랑대,37.619875,127.084106,노원구


In [57]:
import pandas as pd

# 두 개의 데이터프레임이 있다고 가정합니다.
# df_grouped와 grouped_df

# df_grouped에만 있는 인덱스를 찾기
unique_to_df_grouped = df_grouped.index.difference(grouped_df.index)

# grouped_df에만 있는 인덱스를 찾기
unique_to_grouped_df = grouped_df.index.difference(df_grouped.index)

# df_grouped에만 있는 데이터를 추출
df_grouped_unique = df_grouped.loc[unique_to_df_grouped]

# grouped_df에만 있는 데이터를 추출
grouped_df_unique = grouped_df.loc[unique_to_grouped_df]

# 결과 확인
print("df_grouped에만 있는 데이터:")
print(df_grouped_unique)

print("grouped_df에만 있는 데이터:")
print(grouped_df_unique)


df_grouped에만 있는 데이터:
         가중치_합
역명            
둔촌오륜역      2.1
봉은사역       2.8
삼성중앙역      2.8
삼전역        2.8
서울역        2.4
석촌고분역      3.5
석촌역        3.5
선정릉역       2.8
송파나루역      3.5
언주역        2.8
올림픽공원역     2.1
이수         3.0
종합운동장역     4.2
중앙보훈병원역    3.5
한성백제역      2.1
grouped_df에만 있는 데이터:
           위도          경도    구
역명                            
서울  37.553161  126.972684  용산구
